In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
from google.colab import files


#uploaded = files.upload()
# import io
# df2 = pd.read_csv(io.BytesIO(uploaded['Formula1.csv']))


In [0]:
from IPython.display import Image

# Collecting  Data

In [55]:

Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/simple_mlp_mnist.png", width=500, height=250)

In [0]:
#collect MNIST data
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()

In [57]:

print(f'the shape of x_train:{x_train.shape}')
print(f'the shape of x_test{x_test.shape}')

the shape of x_train:(60000, 28, 28)
the shape of x_test(10000, 28, 28)


We can see that each image consists of a 28*28 pixel image and is a handwritten number.Additionally, we should note that MNIST scales from 0 to 255.

# Split the data into train and validated data

After setting up the model, we need to train it with a sample of so-called training data. However, if this procedure proceeds only based on the training data, our model is learning noises in an unwanted manner, which leads our model to show a high level of performance on the training set but a lower level on the newly entered data -overfitting problem.

Therefore, we need to come out with another dataset to evaluate a given model fit on the training set while tunning hyperparameters. This is where validation sets kick in.  This gives guides as to how many iterations should be run before the model is suffering from overfitting.

![image](https://user-images.githubusercontent.com/53164959/75613159-beddef00-5b6d-11ea-9d76-e065c0622e1d.png)


In [58]:
x_val=x_train[50000:60000]
x_train=x_train[0:50000]
y_val=y_train[50000:60000]
y_train=y_train[0:50000]

print(f'x_train samples:{x_train.shape[0]}, x_train image pixels: {x_train.shape[1]}*{x_train.shape[2]}')
print(f'x_val samples:{x_val.shape[0]}, x_val image pixels: {x_val.shape[1]}*{x_val.shape[2]}')

x_train samples:50000, x_train image pixels: 28*28
x_val samples:10000, x_val image pixels: 28*28


# Reshaping Our image data

In order to recoginze the handwritten digits, our very first step is to put the all information to a neural network. This begins as 756 items are now conneted to each input nodes in our model. This indireclty explains why we need to flattenour the shape of column to make the first dimension array.

![image](https://user-images.githubusercontent.com/53164959/75613385-60fed680-5b70-11ea-8102-567f9c2c55fd.png)


In [59]:
x_train=x_train.reshape(50000,784)
x_val=x_val.reshape(10000,784)
x_test=x_test.reshape(10000,784)

print(f'x_train shape: {x_train.shape}')
print(f'x_val shape: {x_val.shape}')
print(f'x_test shape: {x_test.shape}')

x_train shape: (50000, 784)
x_val shape: (10000, 784)
x_test shape: (10000, 784)


# Why Normalization Required?
Having all scaled inputs to a neural network contributes to the conversion of error surface into a more spherical shape. This helps the Gradient Descent converge faster to the global minimum. That is why he mentions the convergence is fast and improved.


In [0]:
x_train=x_train.astype(np.float32)
x_val=x_val.astype(np.float32)
x_test=x_test.astype(np.float32)

#Note that mnist scales from 0 to 255 where 0 indicates there is no hand written.
gray_scale=255
x_train/=gray_scale
x_val/=gray_scale
x_test/=gray_scale


# Creation of Dummy Variable

We need to convert labels into variable taking either 0 or 1. This ca be done through one hot encoding.



In [0]:
num_classes=10
y_train=tf.keras.utils.to_categorical(y_train,num_classes)
y_val=tf.keras.utils.to_categorical(y_val,num_classes)
y_test=tf.keras.utils.to_categorical(y_test,num_classes)


# Construction of Layers in the  MLP 

![image](https://user-images.githubusercontent.com/53164959/75618281-5152b280-5baf-11ea-8aea-d9c410bd7063.png)


In [0]:
tf.compat.v1.disable_eager_execution()
x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])


In [0]:
def mlp(x):
    # hidden layer1
    w1 = tf.Variable(tf.random_uniform([784,256]))
    b1 = tf.Variable(tf.zeros([256]))
    h1 = tf.nn.relu(tf.matmul(x, w1) + b1)
    # hidden layer2
    w2 = tf.Variable(tf.random_uniform([256,128]))
    b2 = tf.Variable(tf.zeros([128]))
    h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)
    # output layer
    w3 = tf.Variable(tf.random_uniform([128,10]))
    b3 = tf.Variable(tf.zeros([10]))
    logits= tf.matmul(h2, w3) + b3
    
    return logits
logits = mlp(x)

In [0]:
loss_op=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=y))

In [0]:
train_op=tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_op)

In [107]:
#intialize
init=tf.global_variables_initializer()

#train_hyperparameter
epoch_cnt=30
batch_size=1000
iteration=len(x_train)//batch_size

#start training
with tf.Session() as sess:
  sess.run(init)
  for epoch in range(epoch_cnt):
    avg_loss=0
    start=0;end=batch_size
    
    #Each for loop amounts to 50. After an iteration, strat and end increase by 1000
    for i in range(iteration):
      _,loss=sess.run([train_op,loss_op],feed_dict={x:x_train[start:end],y:y_train[start:end]})
      start+=batch_size;end+=batch_size
      #compute the average loss
      avg_loss+=loss/iteration
    #After training,we are ready to bring the predicted numbers
    preds=tf.nn.softmax(logits)
    correct=tf.equal(tf.argmax(preds,axis=1),tf.argmax(y,axis=1))
    accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))
    cur_val_acc=accuracy.eval({x:x_val,y:y_val})
    print(f"epoch:{epoch},validation_accuracy:{cur_val_acc},loss:{avg_loss}")
# Test model
  preds = tf.nn.softmax(logits)  # Apply softmax to logits
  correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
  # Calculate accuracy
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
  print("[Test Accuracy] :", accuracy.eval({x: x_test, y: y_test}))
 


epoch:0,validation_accuracy:0.09880000352859497,loss:8542.699897460941
epoch:1,validation_accuracy:0.7709000110626221,loss:399.82943786621087
epoch:2,validation_accuracy:0.8824999928474426,loss:17.521709346771246
epoch:3,validation_accuracy:0.8885999917984009,loss:10.472563209533691
epoch:4,validation_accuracy:0.8982999920845032,loss:8.075983915328978
epoch:5,validation_accuracy:0.8888999819755554,loss:6.677321166992188
epoch:6,validation_accuracy:0.8902000188827515,loss:5.641556367874146
epoch:7,validation_accuracy:0.8967999815940857,loss:5.099739503860472
epoch:8,validation_accuracy:0.8967999815940857,loss:4.386400728225707
epoch:9,validation_accuracy:0.911899983882904,loss:5.494774961471557
epoch:10,validation_accuracy:0.8722000122070312,loss:4.480757584571838
epoch:11,validation_accuracy:0.9122999906539917,loss:6.228302817344666
epoch:12,validation_accuracy:0.8841000199317932,loss:4.498648958206176
epoch:13,validation_accuracy:0.8503000140190125,loss:13.00392523765564
epoch:14,vali